In [4]:
%pylab qt

import os
import itertools
import cv2
import numpy as np
from math import radians, degrees
from glob import glob
from scipy.signal import argrelextrema
from sklearn.preprocessing import normalize

import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline/'

def extract_landmark_points(outline_points, outline_edges):
    step_size = 1
    landmark_definitions = [
        {
            'a_min': 30,
            'a_max': 90,
            'method': 'max'
        },
        {
            'a_min': 80,
            'a_max': 100,
            'method': 'min'
        },
        {
            'a_min': 90,
            'a_max': 150,
            'method': 'max'
        },
        {
            'a_min': 160,
            'a_max': 200,
            'method': 'max'
        },
        {
            'a_min': 210,
            'a_max': 270,
            'method': 'max'
        },
        {
            'a_min': 260,
            'a_max': 280,
            'method': 'min'
        },
        {
            'a_min': 270,
            'a_max': 330,
            'method': 'max'
        }
    ]
    landmarks = []
    rho = 2
    centroid = mean(outline_points, axis=0)
    
    for definition in landmark_definitions:
        intersects_for_angles = []
        method = np.argmax if definition['method'] == 'max' else np.argmin
        
        for angle in range(definition['a_min'], definition['a_max']):
            startpoint = centroid
            endpoint = gh.pol2cart(rho, radians(angle)) + centroid
            
            intersect = None
            for edge in outline_edges:
                p1 = outline_points[edge[0], :]
                p2 = outline_points[edge[1], :]
                
                intersect = gh.seg_intersect(p1, p2, startpoint, endpoint)
                if intersect is not None:
                    break;
            if intersect is None:
                raise Exception('No Intersect.')
            else:
                intersects_for_angles.append(intersect)
        
        intersects_for_angles = np.array(intersects_for_angles)
        distances = np.linalg.norm(intersects_for_angles - np.tile(centroid, (intersects_for_angles.shape[0], 1)), axis=1)
        
        landmarks.append(intersects_for_angles[method(distances), :])
        
    return np.array(landmarks)

def register_outline_with_icp(reference, reference_edges, points, edges, no_iterations = 1):
    landmarks_reference = extract_landmark_points(reference, reference_edges)

    for i in range(no_iterations):
        landmarks = extract_landmark_points(points, edges)
        #Compute the transformation between the current source
        #and destination cloudpoint
        R, t = gh.estimate_rigid_transform(landmarks, landmarks_reference)
        R = np.array(R)
        t = np.array(t)
        #Transform the previous source and update the
        #current source cloudpoint
        points = (np.dot(R, points.transpose())).transpose() + np.tile(t, (points.shape[0], 1))
    
    return points

def do_registration(files):
    loaded = []
    transposed_and_scaled = []
    registered = []
    
    for f in files:
        tri = np.load(f)
        points = tri['points']
        triangles = tri['triangles']
        
        label = os.path.basename(f)
        label = ''.join([i if ord(i) < 128 else ' ' for i in label])
        
        outline_points, outline_edges = gh.extract_outline(points, triangles)
        
        loaded.append({
            'label': label,
            'filename': f,
            'points': outline_points,
            'edges': outline_edges
        })
    
    for outline in loaded:
        points = outline['points']
        
        centroid = mean(points, axis=0)
        points = points - np.tile(centroid, (len(points), 1))
        
        scale = sqrt(np.sum(np.power(points, 2)) / len(points))
        points = np.divide(points, scale)
        points[: ,0] = -points[:, 0]
        points, edges = gh.normalize_outline(points, outline['edges'])
        
        transposed_and_scaled.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    reference = max(transposed_and_scaled, key=lambda o: o['points'].shape[0])
    for outline in transposed_and_scaled:
        points = outline['points']
        edges = outline['edges']
        
        if not np.array_equal(points, reference):
            points = register_outline_with_icp(reference['points'], reference['edges'], points, outline['edges'])
            points, edges = gh.normalize_outline(points, edges)
        registered.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    return registered

Populating the interactive namespace from numpy and matplotlib
Current base path: C:\Users\Stefan\Dropbox\Masterarbeit


`%matplotlib` prevents importing * from pylab and numpy


In [5]:
filenames = glob(DATA_PATH + '*.npz')
registered = do_registration(filenames)

In [7]:
dh.outlines(plt, registered)

(<matplotlib.figure.Figure at 0x2336bb00>,
 <matplotlib.axes._subplots.AxesSubplot at 0x24bc5b38>)

In [3]:
outline = registered[20]
landmarks = extract_landmark_points(outline['points'], outline['edges'])
fig, axes = dh.outline(plt, outline)
axes.plot(landmarks[:, 1], landmarks[:, 0], 'bo')

In [6]:
for outline in registered:
    basename = os.path.basename(outline['filename'])
    destination = os.path.join(TO_PATH, basename)
    
    np.savez(destination, points=outline['points'], edges=outline['edges'])